<br><br>
<h1 align="center"> 04 Data Preparation for ML </h1>
<br><br>

In [45]:
'''Import the libraries'''
import numpy as np
import scipy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML
import hashlib
import joblib
import datetime
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

In [2]:
'''Load the data. It's big, set low_memory=False'''
loan_data = pd.read_csv('/opt/apps/ml-data/bigdata/lending_club/loan_data.csv', low_memory=False)

In [3]:
print(loan_data.shape)

(1338224, 38)


In [5]:
cols = loan_data.columns
missing_value_count = 0
d = []
d_types = loan_data.dtypes
'''Look at the correlation matrix. Sort the matrix values in descending order wrt median_house_value'''
corr_matrix = housing.corr()
corr_matrix = corr_matrix["median_house_value"].sort_values(ascending=False)
print(corr_matrix)
for ind, col in enumerate(cols):
    missing_value_count = loan_data[col].isnull().sum()
    

    d.append({'Columns': col, 'Missing Values': missing_value_count, 'Data Type': d_types[col], 
              'Unique Values': loan_data[col].nunique()})
    missing_value_count = 0
    
    
loan_data_info = pd.DataFrame(d)

display(HTML(loan_data_info.to_html()))  

,Columns,Missing Values,Data Type,Unique Values
0,loan_amnt,0,int64,1558
1,term,0,object,2
2,int_rate,0,float64,654
3,installment,0,float64,83364
4,grade,0,object,7
5,sub_grade,0,object,35
6,emp_length,0,object,11
7,home_ownership,0,object,6
8,annual_inc,0,float64,64147
9,verification_status,0,object,3


In [6]:
'''Drop selected columns, not needed for ML'''
loan_data = loan_data.drop(['earliest_cr_line', 'issue_d', 'loan_status'], axis = 1)

In [7]:
print(loan_data.shape)

(1338224, 35)


In [10]:
'''Create the numeric target variable'''


def create_risk_n(risk_b):
    if risk_b == 'Risk':
        return 1
    elif risk_b == 'No Risk':
        return 0
    

loan_data['risk_n'] = loan_data['Risk'].apply(create_risk_n)

loan_data.reset_index(drop=True, inplace=True)

In [13]:
loan_data['Risk'].value_counts()

No Risk    1041952
Risk        296272
Name: Risk, dtype: int64

In [14]:
loan_data['risk_n'].value_counts()

0    1041952
1     296272
Name: risk_n, dtype: int64

In [15]:
loan_data = loan_data.drop(['Risk'], axis = 1)
loan_data.reset_index(drop=True, inplace=True)

In [16]:
print(loan_data.shape)

(1338224, 35)


In [17]:
cols = loan_data.columns
missing_value_count = 0
d = []
d_types = loan_data.dtypes

for ind, col in enumerate(cols):
    missing_value_count = loan_data[col].isnull().sum()
    

    d.append({'Columns': col, 'Missing Values': missing_value_count, 'Data Type': d_types[col], 
              'Unique Values': loan_data[col].nunique()})
    missing_value_count = 0
    
    
loan_data_info = pd.DataFrame(d)

display(HTML(loan_data_info.to_html()))  

,Columns,Missing Values,Data Type,Unique Values
0,loan_amnt,0,int64,1558
1,term,0,object,2
2,int_rate,0,float64,654
3,installment,0,float64,83364
4,grade,0,object,7
5,sub_grade,0,object,35
6,emp_length,0,object,11
7,home_ownership,0,object,6
8,annual_inc,0,float64,64147
9,verification_status,0,object,3


In [18]:
'''Lets look at No Risk and Risk loans, we need to have the same ratio for the training set nd test set'''
loan_data['risk_n'].value_counts()/len(loan_data['risk_n'])

0    0.778608
1    0.221392
Name: risk_n, dtype: float64

In [20]:
'''Split the data to 80% training set and 20% test set'''
stratified = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_set, test_set in stratified.split(loan_data, loan_data["risk_n"]):
    stratified_train = loan_data.loc[train_set]
    stratified_test = loan_data.loc[test_set]
    
print('Train Set Ratio \n', stratified_train["risk_n"].value_counts()/len(stratified_train))
print('Test Set Ratio \n', stratified_test["risk_n"].value_counts()/len(stratified_test))

Train Set Ratio 
 0    0.778608
1    0.221392
Name: risk_n, dtype: float64
Test Set Ratio 
 0    0.77861
1    0.22139
Name: risk_n, dtype: float64


In [21]:
'''Lets prepare the data'''
train_df = stratified_train
test_df = stratified_test

# Let's Shuffle the data
train_df = train_df.sample(frac=1).reset_index(drop=True)
test_df = test_df.sample(frac=1).reset_index(drop=True)

In [22]:
# Train set (Normal training dataset)
X_train = train_df.drop('risk_n', axis=1)
y_train = train_df['risk_n']


# Test Dataset
X_test = test_df.drop('risk_n', axis=1)
y_test = test_df['risk_n']

In [72]:
X_test.shape

(267645, 34)

In [73]:
'''Separate numeric and categorical features'''
numeric = X_train.select_dtypes(exclude=["object"])
categorical = X_train.select_dtypes(["object"])

In [74]:
num_attribs = list(numeric)
cat_attribs = list(categorical)

In [76]:
len(num_attribs)

22

In [78]:
len(cat_attribs)

12

In [79]:
'''Separate numeric and categorical features'''
numeric_1 = X_test.select_dtypes(exclude=["object"])
categorical_1 = X_test.select_dtypes(["object"])
num_attribs_1 = list(numeric_1)
cat_attribs_1 = list(categorical_1)

In [80]:
len(num_attribs_1)

22

In [81]:
len(cat_attribs_1)

12

In [82]:
class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features as a numeric array.
    The input to this transformer should be a matrix of integers or strings,
    denoting the values taken on by categorical (discrete) features.
    The features can be encoded using a one-hot aka one-of-K scheme
    (``encoding='onehot'``, the default) or converted to ordinal integers
    (``encoding='ordinal'``).
    This encoding is needed for feeding categorical data to many scikit-learn
    estimators, notably linear models and SVMs with the standard kernels.
    Read more in the :ref:`User Guide <preprocessing_categorical_features>`.
    Parameters
    ----------
    encoding : str, 'onehot', 'onehot-dense' or 'ordinal'
        The type of encoding to use (default is 'onehot'):
        - 'onehot': encode the features using a one-hot aka one-of-K scheme
          (or also called 'dummy' encoding). This creates a binary column for
          each category and returns a sparse matrix.
        - 'onehot-dense': the same as 'onehot' but returns a dense array
          instead of a sparse matrix.
        - 'ordinal': encode the features as ordinal integers. This results in
          a single column of integers (0 to n_categories - 1) per feature.
    categories : 'auto' or a list of lists/arrays of values.
        Categories (unique values) per feature:
        - 'auto' : Determine categories automatically from the training data.
        - list : ``categories[i]`` holds the categories expected in the ith
          column. The passed categories are sorted before encoding the data
          (used categories can be found in the ``categories_`` attribute).
    dtype : number type, default np.float64
        Desired dtype of output.
    handle_unknown : 'error' (default) or 'ignore'
        Whether to raise an error or ignore if a unknown categorical feature is
        present during transform (default is to raise). When this is parameter
        is set to 'ignore' and an unknown category is encountered during
        transform, the resulting one-hot encoded columns for this feature
        will be all zeros.
        Ignoring unknown categories is not supported for
        ``encoding='ordinal'``.
    Attributes
    ----------
    categories_ : list of arrays
        The categories of each feature determined during fitting. When
        categories were specified manually, this holds the sorted categories
        (in order corresponding with output of `transform`).
    Examples
    --------
    Given a dataset with three features and two samples, we let the encoder
    find the maximum value per feature and transform the data to a binary
    one-hot encoding.
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    See also
    --------
    sklearn.preprocessing.OneHotEncoder : performs a one-hot encoding of
      integer ordinal features. The ``OneHotEncoder assumes`` that input
      features take on values in the range ``[0, max(feature)]`` instead of
      using the unique values.
    sklearn.feature_extraction.DictVectorizer : performs a one-hot encoding of
      dictionary items (also handles string-valued features).
    sklearn.feature_extraction.FeatureHasher : performs an approximate one-hot
      encoding of dictionary items or strings.
    """

    
    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

        
    def fit(self, X, y=None):
        """Fit the CategoricalEncoder to X.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_feature]
            The data to determine the categories of each feature.
        Returns
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("encoding should be either 'onehot', 'onehot-dense' "
                        "or 'ordinal', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("handle_unknown should be either 'error' or "
                        "'ignore', got %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("handle_unknown='ignore' is not supported for"
                             " encoding='ordinal'")

        X = check_array(X, dtype=np.object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Found unknown categories {0} in column {1}"
                               " during fit".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    
    def transform(self, X):
        """Transform X using one-hot encoding.
        Parameters
        ----------
        X : array-like, shape [n_samples, n_features]
            The data to encode.
        Returns
        -------
        X_out : sparse matrix or a 2-d array
            Transformed input.
        """
        X = check_array(X, accept_sparse='csc', dtype=np.object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=np.int)
        X_mask = np.ones_like(X, dtype=np.bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Found unknown categories {0} in column {1}"
                           " during transform".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Set the problematic rows to an acceptable value and
                    # continue `The rows are marked `X_mask` and will be
                    # removed later.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

In [89]:
'''
Transformation and Feature Scaling:
All transformation pipelines in one place.
'''
class DataFrameSelector1(BaseEstimator, TransformerMixin):
    
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

    
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names]


class CustomLabelBinarizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, sparse_output=False):
          self.sparse_output = sparse_output
    
    def fit(self, X, y=None):
        self.enc = LabelBinarizer(sparse_output=self.sparse_output)
        self.enc.fit(X)
        return self
    
    def transform(self, X, y=None):
          return self.enc.transform(X)
        

class CustomBinarizer(BaseEstimator, TransformerMixin):
    
    def __init__(self, sparse_output=False):
          self.sparse_output = sparse_output
    
    def fit(self, X, y=None):
        self.label_binarizer_ = LabelBinarizer(sparse_output=self.sparse_output).fit(X)
        return self
    
    def transform(self, X):
        return self.label_binarizer_.transform(X)


num_pipeline = Pipeline([
                        ('selector', DataFrameSelector(num_attribs)),
                        ('std_scaler', StandardScaler()),
                        ])

cat_pipeline = Pipeline([
                        ('selector', DataFrameSelector(cat_attribs)),
                        ('encoder', CategoricalEncoder(encoding="ordinal")),
                        ])

full_pipeline = FeatureUnion(transformer_list=[
                            ("num_pipeline", num_pipeline),
                            ("cat_pipeline", cat_pipeline),
                            ])

In [90]:
'''Transform the data using pipeline'''
print("Raw Data: ", X_train.shape)
X_train_prepared = full_pipeline.fit_transform(X_train)
print("Prepared Data: ", X_train_prepared.shape)

Raw Data:  (1070579, 34)
Prepared Data:  (1070579, 34)


In [91]:
'''Transform the data using pipeline'''
print("Raw Data: ", X_test.shape)
X_test_prepared = full_pipeline.fit_transform(X_test)
print("Prepared Data: ", X_test_prepared.shape)

Raw Data:  (267645, 34)
Prepared Data:  (267645, 34)


In [93]:
X_test_prepared

array([[-1.20145047e+00, -3.75225991e-01, -1.17547994e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.73380030e-01,  3.01444014e-04, -2.87759858e-03, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 4.49173735e-01,  2.38135486e-01,  7.38061864e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 9.37507194e-01,  2.43288472e+00,  1.76034521e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.72990371e-01, -1.35305685e-01, -9.11831840e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 8.54690408e-01,  1.59837931e+00,  5.86829536e-01, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [94]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
# log_reg_sm = LogisticRegression()
log_reg.fit(X_train_prepared, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [95]:
from sklearn.metrics import accuracy_score

normal_ypred = log_reg.predict(X_test_prepared)
print(accuracy_score(y_test, normal_ypred))

0.7812699658129239
